In [1]:
!pip install pandas numpy scikit-learn matplotlib seaborn nltk tqdm


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
print(df.columns)

Index(['comment', 'film_name', 'point'], dtype='object')


In [10]:
import pandas as pd

# CSV dosyasını yükle
df = pd.read_csv("/content/drive/MyDrive/Colab nlp proje/turkish_movie_sentiment_dataset.csv")

# 'point' sütunundaki virgülleri noktaya çevir, sonra float'a dönüştür
df['point'] = df['point'].astype(str).str.replace(',', '.')
df['point'] = df['point'].astype(float)

# Yeni etiket sütunu oluştur (puan 3 ve üzeriyse pozitif, değilse negatif)
df['sentiment'] = df['point'].apply(lambda x: 'positive' if x >= 3 else 'negative')

# İlk 5 satıra ve sınıf dağılımına göz atalım
print(df[['comment', 'point', 'sentiment']].head())
print(df['sentiment'].value_counts())


                                             comment  point sentiment
0  \n                      Jean Reno denince zate...    5.0  positive
1  \n                      Ekşın falan izlemek is...    5.0  positive
2  \n                      Bu yapım hakkında öyle...    5.0  positive
3  \n                      finali yeter... (sting...    5.0  positive
4  \n                      Jean Reno..\r\nbu adam...    5.0  positive
sentiment
positive    57830
negative    25397
Name: count, dtype: int64


In [11]:
import re

def clean_text(text):
    text = str(text).lower()  # küçük harfe çevir
    text = re.sub(r"http\S+", "", text)  # URL'leri kaldır
    text = re.sub(r"[^a-zçğıöşü0-9\s]", "", text)  # noktalama işaretlerini temizle
    text = re.sub(r"\s+", " ", text).strip()  # fazla boşlukları temizle
    return text


In [12]:
# Temizlenmiş yorumları yeni bir sütuna yaz
df['clean_text'] = df['comment'].apply(clean_text)

# Kontrol edelim
print(df[['comment', 'clean_text']].head())


                                             comment  \
0  \n                      Jean Reno denince zate...   
1  \n                      Ekşın falan izlemek is...   
2  \n                      Bu yapım hakkında öyle...   
3  \n                      finali yeter... (sting...   
4  \n                      Jean Reno..\r\nbu adam...   

                                          clean_text  
0  jean reno denince zaten leon filmi gelir akla ...  
1  ekşın falan izlemek istiyorsanız eğer bunu izl...  
2  bu yapım hakkında öyle çok şey yazabilirim ki ...  
3  finali yeter sting shape of my heart bazıları ...  
4  jean reno bu adam kusursuz biri ve oyunculugu ...  


In [13]:
from sklearn.model_selection import train_test_split

# Veriyi eğitim ve test setlerine ayıralım
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['sentiment'], test_size=0.2, random_state=42)

# Boyutları kontrol edelim
print(len(X_train), len(X_test))


66581 16646


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörlemesi
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)  # 1-2 gram ve 5000 en sık kelime
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Sonuçları kontrol edelim
print(X_train_vec.shape)


(66581, 5000)


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Naive Bayes modelini oluştur ve eğit
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Eğitim ve test setlerinde tahminler yapalım
y_pred_train = model.predict(X_train_vec)
y_pred_test = model.predict(X_test_vec)

# Doğruluk oranı
print(f"Train Accuracy: {accuracy_score(y_train, y_pred_train)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test)}")


Train Accuracy: 0.804013156906625
Test Accuracy: 0.8002523128679563


In [19]:
from sklearn.metrics import classification_report, accuracy_score

# Eğitim ve test setlerinde tahminler yapalım
y_pred_train = model.predict(X_train_vec)
y_pred_test = model.predict(X_test_vec)

# Doğruluk oranı
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Değerlendirme raporu
report = classification_report(y_test, y_pred_test, target_names=['Negatif', 'Pozitif'], output_dict=True)

# Türkçe rapor
print(f"Eğitim Doğruluğu: %{train_accuracy * 100:.2f}")
print(f"Test Doğruluğu: %{test_accuracy * 100:.2f}")

print("\nTest Set Değerlendirmesi:")
print(f"Negatif sınıf (Precision): {report['Negatif']['precision']:.2f}")
print(f"Negatif sınıf (Recall): {report['Negatif']['recall']:.2f}")
print(f"Negatif sınıf (F1-Skoru): {report['Negatif']['f1-score']:.2f}")
print(f"Pozitif sınıf (Precision): {report['Pozitif']['precision']:.2f}")
print(f"Pozitif sınıf (Recall): {report['Pozitif']['recall']:.2f}")
print(f"Pozitif sınıf (F1-Skoru): {report['Pozitif']['f1-score']:.2f}")

print(f"\nGenel Doğruluk (Accuracy): {report['accuracy']:.2f}")
print(f"Macro Ortalama F1-Skoru: {report['macro avg']['f1-score']:.2f}")
print(f"Ağırlıklı Ortalama F1-Skoru: {report['weighted avg']['f1-score']:.2f}")


Eğitim Doğruluğu: %80.40
Test Doğruluğu: %80.03

Test Set Değerlendirmesi:
Negatif sınıf (Precision): 0.81
Negatif sınıf (Recall): 0.47
Negatif sınıf (F1-Skoru): 0.59
Pozitif sınıf (Precision): 0.80
Pozitif sınıf (Recall): 0.95
Pozitif sınıf (F1-Skoru): 0.87

Genel Doğruluk (Accuracy): 0.80
Macro Ortalama F1-Skoru: 0.73
Ağırlıklı Ortalama F1-Skoru: 0.78


In [20]:
from sklearn.model_selection import GridSearchCV

# Parametrelerin grid'ini tanımlayalım
param_grid = {
    'alpha': [0.1, 0.5, 1, 1.5, 2],  # Düzenleme parametresi
    'fit_prior': [True, False]  # Sınıf önceliği kullanılıp kullanılmayacağı
}

# Naive Bayes modelini tanımlayalım
model = MultinomialNB()

# GridSearchCV'yi tanımlayalım
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Eğitim verisi ile optimize edelim
grid_search.fit(X_train_vec, y_train)

# En iyi parametreleri ve sonucu yazdıralım
print("En İyi Parametreler:", grid_search.best_params_)
print("En İyi Sonuç:", grid_search.best_score_)

# En iyi model ile test doğruluğunu kontrol edelim
best_model = grid_search.best_estimator_
y_pred_test_optimized = best_model.predict(X_test_vec)
print(f"Test Doğruluğu (Optimized): {accuracy_score(y_test, y_pred_test_optimized)}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
En İyi Parametreler: {'alpha': 0.5, 'fit_prior': True}
En İyi Sonuç: 0.7967438239659744
Test Doğruluğu (Optimized): 0.80067283431455


In [23]:
# Veriden rastgele bir örnek alalım (Örneğin, 10. satırı)
example_text = df['comment'].iloc[80]
example_label = df['sentiment'].iloc[80]

# Veriyi vektörize edelim (Bu, eğitim verisi ile yaptığımız gibi olmalı)
example_vectorized = vectorizer.transform([example_text])

# Model tahminini alalım
predicted_label = best_model.predict(example_vectorized)

# Tahminin sonuçlarını yazdıralım
print(f"Örnek Yorum: {example_text}")
print(f"Gerçek Etiket: {example_label}")
print(f"Modelin Tahmini: {predicted_label[0]}")


Örnek Yorum: 
                      Sinema tarihinde bazı filmler 'overrated' bazıları 'underrated' kategorisne girer. Inception(başlangıç kel alaka bir başlık olmuş film için) kesinlikle overrated yani olduğundan fazla değer verilen bir film. Fim izlemeden önce yorumlara bakınca neredeyse gelmiş geçmiş en iyi film demeye varacak kadar gözü kapalı beğenen sinemaseverler var. Araç sallanınca rüyaların da sallanması gibi abuk  mantıklı defoların olmasına rağmen üzerinde çok çalışılmış harika bir senaryosu var. Ne var ki söz konusu 'rüya' olunca efekt açısında kısır buldum. rüyalardaki sınırsız potansiyeli film yansıtmamaış Çok daha muazzam efektler olmalıydı. Bence filmin en önemli kusuru 'şok ' edici bir etkisi olmamasıydı. Sanıldığının aksine daha ilk dakikada konu veriliyor matrix de yaşadığımız şok etkisinin 5 de 1inin yaşamıyoruz. Dediğim gibi değeri abartılmış bir film...
        
            
Gerçek Etiket: negative
Modelin Tahmini: positive
